In [1]:
import os
import pandas as pd

In [2]:
res = pd.read_csv('../results/eurosat_probing_rebuttal_test.csv', index_col = 0)
res['Data Set'] = 'EuroSat'
res['Dimension'] = res['N Shots'] - 1
all_results = res

datasets = {
            'Flowers102': 'flowers',
            'CIFAR-10': 'cifar10',
            }

base_path = '../results'
suffix = '_probing_rebuttal_test.csv'
for d_name in datasets.keys():
    d_id = datasets[d_name]
    path = os.path.join(base_path,f'{d_id}{suffix}')
    res = pd.read_csv(path, index_col = 0)
    res['Data Set'] = d_name
    if d_id in ['flowers', 'cifar10']:
        res['Dimension'] = 1
    else:
        res['Dimension'] = res['N Shots'] - 1
    all_results = pd.concat([all_results, res])

In [3]:


# Group by 'N Shots', 'Method', and 'Data Set' and calculate mean and std
summary_df = all_results.groupby(['N Shots', 'Method', 'Data Set', 'Dimension'])['Accuracy'].agg(['mean', 'std']).reset_index()

# Rename columns for clarity
summary_df.rename(columns={'mean': 'Accuracy Mean', 'std': 'Accuracy Std'}, inplace=True)

summary_df = summary_df.sort_values(by = ['Data Set', 'N Shots', 'Method'])

# Display the result
print(summary_df)


    N Shots           Method    Data Set  Dimension  Accuracy Mean  \
0         3         FlagNets    CIFAR-10          1        0.59601   
3         3        ProtoNets    CIFAR-10          1        0.58432   
6         3     ProtoNetsMod    CIFAR-10          1        0.58608   
9         3     SubspaceNets    CIFAR-10          1        0.58527   
12        3  SubspaceNetsMod    CIFAR-10          1        0.59634   
15        5         FlagNets    CIFAR-10          1        0.65173   
18        5        ProtoNets    CIFAR-10          1        0.64460   
21        5     ProtoNetsMod    CIFAR-10          1        0.65161   
24        5     SubspaceNets    CIFAR-10          1        0.63779   
27        5  SubspaceNetsMod    CIFAR-10          1        0.65184   
30        7         FlagNets    CIFAR-10          1        0.67999   
33        7        ProtoNets    CIFAR-10          1        0.67886   
36        7     ProtoNetsMod    CIFAR-10          1        0.68634   
39        7     Subs

In [4]:
# Define the custom order for 'Data Set'
data_set_order = ['EuroSat', 'CIFAR-10', 'Flowers102']

# Convert 'Data Set' to a categorical type with the specified order
summary_df['Data Set'] = pd.Categorical(summary_df['Data Set'], categories=data_set_order, ordered=True)


# for n_shots in summary_df['N Shots'].unique():
#     idx = summary_df['N Shots'] == n_shots
pivot_df = summary_df.pivot_table(
    columns= 'Method',
    index= ['N Shots', 'Data Set'],
    values='Accuracy Mean'
)
pivot_df = pivot_df.round(3)
pivot_df = pivot_df.applymap(lambda x: f"{x:.3f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')
print(pivot_df)
print(pivot_df.to_latex(index=True))

/tmp/ipykernel_26171/317577426.py:10: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = summary_df.pivot_table(
/tmp/ipykernel_26171/317577426.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  pivot_df = pivot_df.applymap(lambda x: f"{x:.3f}".rstrip('0').rstrip('.') if pd.notnull(x) else '')


Method             FlagNets ProtoNets ProtoNetsMod SubspaceNets  \
N Shots Data Set                                                  
3       EuroSat       0.777     0.759        0.767        0.768   
        CIFAR-10      0.596     0.584        0.586        0.585   
        Flowers102    0.902     0.879        0.882        0.888   
5       EuroSat       0.818     0.798        0.807        0.808   
        CIFAR-10      0.652     0.645        0.652        0.638   
        Flowers102    0.932     0.911        0.914         0.92   
7       EuroSat       0.839     0.817        0.826        0.829   
        CIFAR-10       0.68     0.679        0.686        0.667   
        Flowers102    0.945     0.923        0.927        0.934   

Method             SubspaceNetsMod  
N Shots Data Set                    
3       EuroSat              0.776  
        CIFAR-10             0.596  
        Flowers102           0.902  
5       EuroSat              0.818  
        CIFAR-10             0.652  
   

In [5]:
# Define the custom order for 'Data Set'
data_set_order = ['EuroSat', 'CIFAR-10', 'Flowers102']

idx = (summary_df['Data Set'] == 'EuroSat') |\
      (summary_df['Data Set'] == 'CIFAR-10') |\
      (summary_df['Data Set'] == 'Flowers102')
summary_df = summary_df[idx]

# Convert 'Data Set' to a categorical type with the specified order
summary_df['Data Set'] = pd.Categorical(summary_df['Data Set'], categories=data_set_order, ordered=True)

# Multiply the 'Accuracy Mean' and 'Accuracy Std' by 100
summary_df['Accuracy Mean'] *= 100
summary_df['Accuracy Std'] *= 100

# Pivot the table with 'Accuracy Mean' and 'Accuracy Std' separately
pivot_mean = summary_df.pivot_table(
    columns='Method',
    index=['N Shots', 'Data Set'],
    values='Accuracy Mean'
)

pivot_std = summary_df.pivot_table(
    columns='Method',
    index=['N Shots', 'Data Set'],
    values='Accuracy Std'
)

pivot_combined = pivot_mean.copy()  # Start with a copy of the mean table for structure
for col in pivot_combined.columns:
    pivot_combined[col] = [
        f"{mean:.1f} ± {std:.1f}" if pd.notnull(mean) and pd.notnull(std) else ''
        for mean, std in zip(pivot_mean[col], pivot_std[col])
    ]

print(pivot_combined)
print(pivot_combined.to_latex(index=True))

Method                FlagNets   ProtoNets ProtoNetsMod SubspaceNets  \
N Shots Data Set                                                       
3       EuroSat     77.7 ± 1.0  75.9 ± 0.9   76.7 ± 1.0   76.8 ± 1.1   
        CIFAR-10    59.6 ± 1.0  58.4 ± 0.8   58.6 ± 0.9   58.5 ± 0.9   
        Flowers102  90.2 ± 0.7  87.9 ± 0.9   88.2 ± 1.0   88.8 ± 0.8   
5       EuroSat     81.8 ± 0.7  79.8 ± 0.8   80.7 ± 0.8   80.8 ± 0.8   
        CIFAR-10    65.2 ± 0.9  64.5 ± 1.0   65.2 ± 0.9   63.8 ± 0.9   
        Flowers102  93.2 ± 0.5  91.1 ± 0.6   91.4 ± 0.6   92.0 ± 0.5   
7       EuroSat     83.9 ± 0.8  81.7 ± 0.8   82.6 ± 0.8   82.9 ± 0.8   
        CIFAR-10    68.0 ± 0.7  67.9 ± 0.8   68.6 ± 0.8   66.7 ± 0.7   
        Flowers102  94.5 ± 0.5  92.3 ± 0.5   92.7 ± 0.5   93.4 ± 0.5   

Method             SubspaceNetsMod  
N Shots Data Set                    
3       EuroSat         77.6 ± 1.0  
        CIFAR-10        59.6 ± 1.0  
        Flowers102      90.2 ± 0.7  
5       EuroSat       

/tmp/ipykernel_26171/3211811896.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_mean = summary_df.pivot_table(
/tmp/ipykernel_26171/3211811896.py:23: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_std = summary_df.pivot_table(


In [10]:
summary_df

,N Shots,Method,Data Set,Dimension,Accuracy Mean,Accuracy Std,Accuracy
0,3,FlagNets,CIFAR-10,1,59.158,0.751613,59.16 ± 0.75
5,3,ProtoNets,CIFAR-10,1,58.432,0.830305,58.43 ± 0.83
10,3,SubspaceNets,CIFAR-10,1,58.527,0.892171,58.53 ± 0.89
15,5,FlagNets,CIFAR-10,1,65.138,0.859251,65.14 ± 0.86
20,5,ProtoNets,CIFAR-10,1,64.460,1.033767,64.46 ± 1.03
25,5,SubspaceNets,CIFAR-10,1,63.779,0.928779,63.78 ± 0.93
30,7,FlagNets,CIFAR-10,1,68.414,0.719301,68.41 ± 0.72
35,7,ProtoNets,CIFAR-10,1,67.886,0.803678,67.89 ± 0.80
40,7,SubspaceNets,CIFAR-10,1,66.739,0.733484,66.74 ± 0.73
1,3,FlagNets,EuroSat,2,77.443,1.036863,77.44 ± 1.04


/tmp/ipykernel_73367/2892759902.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_df = summary_df.pivot_table(


TypeError: agg function failed [how->mean,dtype->object]